
include a dynamic lib?



clang -g -O0 -fmodules -x objective-c -fno-objc-arc -c main.c -o main.o

clang main.o -framework Cocoa -dynamiclib -rdynamic -o sog.dylib



In [ ]:
const {
  equal
} = require('assert')
const {
  load,
  DataType,
  open,
  close,
  arrayConstructor,
  define
} = require('ffi-rs')
const path = require('path')
const {platform} = require('os')
const a = 1
const b = 100
const dynamicLib = platform === 'win32' ? '../sog.dll' : (platform === 'linux' ? '../sog.so' : '../sog.dylib')
// First open dynamic library with key for close
const IGNORE_PROGRAMS = [
  'Window Server',
  'Macs Fan Control',
  'Control Center',
  'Spotlight',
  'Dock',
  'SSMenuAgent',
  'Menubar'
]
function enumWindows() {
  // It only needs to be opened once.
  open({
    library: 'sog', // key
    path: path.join(__dirname, dynamicLib) // path
  })
  const count = load({
    library: "sog", // path to the dynamic library file
    funcName: 'enumWindows', // the name of the function to call
    retType: DataType.I32, // the return value type
    paramsType: [], // the parameter types
    paramsValue: [] // the actual parameter values
    // freeResultMemory: true, // whether or not need to free the result of return value memory automatically, default is false
  })
  let windows = []
  for(let i = 0; i < count; i++) {
    const result = load({
      library: "sog", // path to the dynamic library file
      funcName: 'getTitle', // the name of the function to call
      retType: DataType.String, // the return value type
      paramsType: [DataType.I32], // the parameter types
      paramsValue: [i] // the actual parameter values
      // freeResultMemory: true, // whether or not need to free the result of return value memory automatically, default is false
    })
    windows.push(result.split(/:\s/g)[1])
  }
  // Release library memory when you're not using it.
  close('sog')
  return windows.filter((a, i, arr) => arr.indexOf(a) == i && !IGNORE_PROGRAMS.includes(a))
}

module.exports = enumWindows


In [ ]:

// Use define function to define a function signature
const res = define({
  sum: {
      library: "libsum",
      retType: DataType.I32,
      paramsType: [DataType.I32, DataType.I32],
  },
  atoi: {
      library: "libnative",
      retType: DataType.I32,
      paramsType: [DataType.String],
  }
})
equal(res.sum([1, 2]), 3)
equal(res.atoi(["1000"]), 1000)



move mouse


In [ ]:
const {
  equal
} = require('assert')
const {
  load,
  DataType,
  open,
  close,
  arrayConstructor,
  define
} = require('ffi-rs')
const path = require('path')
const {platform} = require('os')

const dynamicLib = platform === 'win32' ? '../mouse.dll' : (platform === 'linux' ? '../mouse.so' : '../mouse.dylib')

function moveMouse(x, y) {
  open({
    library: 'mouse', // key
    path: path.join(__dirname, dynamicLib) // path
  })
  load({
    library: "mouse", // path to the dynamic library file
    funcName: 'SetMousePosition', // the name of the function to call
    retType: DataType.Void, // the return value type
    paramsType: [DataType.Float, DataType.Float], // the parameter types
    paramsValue: [x || 0, y || 0] // the actual parameter values
    // freeResultMemory: true, // whether or not need to free the result of return value memory automatically, default is false
  })
  close('mouse')
}

module.exports = moveMouse



get c parameters?


In [ ]:
const selectCode = importer.import('select antlr tree')

async function getParameters(code) {
    var result = await (await selectCode)(['//functionDefinition', './declarator//declaratorid/@strdata'], code, 'cpp')
    return result
}

module.exports = getParameters




get c exports?


In [ ]:
const selectCode = importer.import('select antlr tree')

async function getExports(code) {
    var result = await (await selectCode)(['//functionDefinition', './declarator//declaratorid/@strdata'], code, 'cpp')
    return result.map(e => typeof e == 'string' ? e : e[0])
}

module.exports = getExports




get c types?


In [ ]:
const selectCode = importer.import('select antlr tree')

async function getTypes(code, functionName) {
    if(functionName) {
        let result = await (await selectCode)(['(//functionDefinition[contains(.//declaratorid/@strdata, "' + functionName + '")]/declSpecifierSeq/declSpecifier/typeSpecifier|//functionDefinition[contains(.//declaratorid/@strdata, "' + functionName + '")]/declarator//typeSpecifier)', './@strdata'], code, 'cpp')
        return result
    }

    var result = await (await selectCode)(['(//functionDefinition/declSpecifierSeq/declSpecifier/typeSpecifier|//functionDefinition/declarator//typeSpecifier)', './@strdata'], code, 'cpp')
    return result
}

module.exports = getTypes

